In [126]:
import pandas as pd
import requests
import json
import numpy as np
from tqdm.auto import tqdm
import datetime

# Import CANVAS_API_KEY from the .env file
from dotenv import load_dotenv
load_dotenv(verbose=True)

import os
API_KEY = os.getenv("CANVAS_API_KEY")
BASE_URL = os.getenv("CANVAS_BASE_URL")

In [ ]:
def my_request(API_URL):
    headers = {'Authorization': 'Bearer '+ API_KEY}
    r = requests.get(BASE_URL+API_URL, headers=headers)
    return r.json()

def get_required_submissions_ids(course_id) : 
    '''extracts the required course ids for the course returns two dataframes:
    - required_assignments
    - capstones
    each with two columns 'id' and 'name' 
    example usage: 
        required_assignments, capstones = get_required_submissions_ids(course_id) '''
    
    r = my_request('/api/v1/courses/{}/assignments?per_page=200'.format(course_id))
    all_assignments = pd.DataFrame(r)
    required_assignments = all_assignments[all_assignments.name.str.contains('Assignment')]
    required_assignments = required_assignments[~required_assignments.name.str.contains('Capstone')]
    capstones = all_assignments[all_assignments.name.str.contains('Capstone')]
    return required_assignments[['id','name']], capstones[['id','name']]

# need to get only active students!! course 1682 failing at student 72758
def get_list_of_students(course_id) : 
    '''get the list of students in a panda series.
    Excample usage:
    student_ids = get_list_of_students(course_id)'''
    
    r = my_request('/api/v1/courses/{}/enrollments?type[]=StudentEnrollment&state[]=active&per_page=200'.format(course_id))
    
    return pd.DataFrame.from_dict(r)['user_id']


def check_student_passing_criteria(student_id, course_id, required_assignments, capstones) :
    '''checks if the student passed or failed the course. returns boolean'''
    r = my_request('/api/v1/courses/{}/students/submissions?student_ids[]={}&per_page=200'.format(course_id,student_id))
    student_submissions = pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']] #could reduce to 'assignment_id' and 'grade'
    student_submissions['grade'].replace(to_replace='complete',value=1, inplace=True)
    student_submissions['grade'].replace(to_replace='incomplete',value=0, inplace=True)
    student_submissions['grade'].fillna(0,inplace=True)
    student_submissions['grade'] = student_submissions['grade'].astype(int) #convert boolean to int
    #student_submissions['late'] = student_submissions['late'].astype(int) # not needed as of right now. could check if too late...
    #student_submissions['missing'] = student_submissions['missing'].astype(int)
    
    capstone_check = pd.merge(student_submissions, capstones, left_on='assignment_id', right_on='id', how='inner')['grade'].sum()
    if capstone_check >= 1 : 
        # meets the capstone passing criteria
        # checking the next criteria
        assignment_check = pd.merge(student_submissions, required_assignments, left_on='assignment_id', right_on='id', how='inner')['grade'].sum()
        if assignment_check >= 7 :
            # meets both criterias --> pass
            return True
    else :
        # didn't meet at least one criteria
        return False

# calculate pass fail for entire course
def check_passing_criteria_from_course_id(course_id, verbose=0) :
    '''gets student pass fail for entire course'''
    outcomes = pd.DataFrame(columns = ['student_id', 'course_id', 'outcome'])
    # get list of required assignments
    print('Getting list of required assignments...', end=" ")
    required_assignments, capstones = get_required_submissions_ids(course_id)
    print('DONE')
    
    # get list of students
    print('Getting list of students...', end=" ")
    student_ids = get_list_of_students(course_id)
    print('DONE')
    
    print('Checking every single student...')
    for s_id in student_ids :
        outcome = check_student_passing_criteria(s_id, course_id, required_assignments, capstones)
        outcomes = outcomes.append({'student_id' : s_id, 'course_id':course_id, 'outcome':outcome},ignore_index=True)
        if verbose == 1 :
            print(s_id, end=" ")
            print(outcome)
        else :
            print("|", end="")
    print(' DONE\n')
        
    return outcomes

In [205]:
r = my_request('/api/v1/courses?enrollment_type=teacher')

response= pd.DataFrame.from_dict(r)
course_ids = response[response['name'].str.contains('AI')]['id']


```
id	name
1116	AI-1
1202	AI-2
1394	AI-3
1574	AI-4
1682	AI-5
```

## Create Outcome Variable: check pass fail for the entire course

In [24]:
outcomes = pd.DataFrame(columns = ['student_id', 'course_id', 'outcome'])

In [25]:
for c in course_ids :
    outcomes = outcomes.append(check_passing_criteria_from_course_id(c, verbose=0))

Getting list of required assignments... DONE
Getting list of students... DONE
Checking every single student...

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| DONE

Getting list of required assignments... DONE
Getting list of students... DONE
Checking every single student...

||||||||||

KeyboardInterrupt: 

In [26]:
outcomes.head()

,student_id,course_id,outcome
0,47649,1116,True
1,52263,1116,True
2,24036,1116,True
3,52137,1116,True
4,52267,1116,True


In [27]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   student_id  100 non-null    object
 1   course_id   100 non-null    object
 2   outcome     99 non-null     object
dtypes: object(3)
memory usage: 3.1+ KB


In [28]:
outcomes.to_csv('../data/outcomes.csv')

### Get student summaries (not useful)

In [31]:
#Get student_summaries
course_id = 1574
r = my_request('/api/v1/courses/{}/analytics/student_summaries?per_page=200'.format(course_id))

response= pd.DataFrame.from_dict(r)

In [159]:
#response

In [33]:
tardiness_summary = pd.DataFrame(list(response.tardiness_breakdown))
student_summaries = response.merge(tardiness_summary, left_index=True, right_index=True).drop(['tardiness_breakdown'], axis='columns')
student_summaries.head()

,id,page_views,max_page_views,page_views_level,participations,max_participations,participations_level,missing,late,on_time,floating,total
0,75494,1618,7132,2,51,51,3,30,5,26,32,93
1,74581,1810,7132,2,40,51,3,47,0,30,16,93
2,56876,555,7132,1,42,51,3,32,5,26,30,93
3,74755,1738,7132,2,14,51,1,66,0,11,16,93
4,74578,4609,7132,3,34,51,2,55,1,21,16,93


This Student Summary contains useful summary info, but it is not in a timeseries fashion. I need a way to find the summary of the first 2,3,4 weeks of data only, not the entire course. The ```max_*``` coluns can be dropped , as well as the ```total``` column

### Find Cut-off date for each course

In [233]:
# get cutoff date for course
def get_cutoff_date(course_id, cutoff_assignment = 'Assignment 4.1'):
                                     
    r = my_request('/api/v1/courses/{}/assignments?per_page=200'.format(course_id))
    df = pd.json_normalize(r)
    date_df = df[df['name'].str.startswith(cutoff_assignment)]
    cutoff_date = date_df.iloc[0]['discussion_topic.posted_at']
    #print(cutoff_date)
    pd.to_datetime(cutoff_date)
    cutoff_date = pd.to_datetime(cutoff_date)+datetime.timedelta(days=7)
    return cutoff_date

In [234]:
cutoff_dates_df=pd.DataFrame()
for c in course_ids :
    cutoff_dates_df = cutoff_dates_df.append({'course_id':c, 'cutoff_date':get_cutoff_date(c)}, ignore_index=True)

In [235]:
cutoff_dates_df = cutoff_dates_df.set_index('course_id')

In [236]:
cutoff_dates_df

,cutoff_date
course_id,
1116.0,2020-01-23 06:39:08+00:00
1202.0,2020-04-01 07:24:32+00:00
1394.0,2020-06-25 07:12:21+00:00
1574.0,2020-09-04 11:32:29+00:00
1682.0,2020-11-04 13:02:37+00:00


### Getting Student History

In [237]:
# Get student history
# let's look at id: 75494
course_id = '1574'
student_id = '75494'
cutoff_date = cutoff_dates_df.iloc[-2]['cutoff_date']

r = my_request('/api/v1/courses/{}/analytics/users/{}/activity?per_page=200'.format(course_id,student_id))


In [238]:
df =pd.DataFrame.from_dict((r['page_views']),orient='index')
df.index = pd.to_datetime(df.index)
df.columns = ['page_views']
df.head()

,page_views
2020-08-03 20:00:00+00:00,19
2020-08-03 23:00:00+00:00,10
2020-08-04 14:00:00+00:00,13
2020-08-05 12:00:00+00:00,10
2020-08-05 13:00:00+00:00,9


In [241]:
df[:cutoff_date]

,page_views
2020-08-03 20:00:00+00:00,19
2020-08-03 23:00:00+00:00,10
2020-08-04 14:00:00+00:00,13
2020-08-05 12:00:00+00:00,10
2020-08-05 13:00:00+00:00,9
2020-08-05 14:00:00+00:00,54
2020-08-08 23:00:00+00:00,2
2020-08-09 21:00:00+00:00,2
2020-08-09 22:00:00+00:00,23
2020-08-10 13:00:00+00:00,16


In [36]:
pd.DataFrame(list(zip(list(r['page_views']),r['page_views'].values())),columns=['date', 'pageviews']).head()

,date,pageviews
0,2020-08-03T20:00:00Z,19
1,2020-08-03T23:00:00Z,10
2,2020-08-04T14:00:00Z,13
3,2020-08-05T12:00:00Z,10
4,2020-08-05T13:00:00Z,9


From this time-series we can extract several features:
- total pageviews (until certain date)
- average pageview / day (until certain date)
- avg/week (until certain date)
- # of hours with page views (until certain date)
- max pageviews/hour (until certain date)
- avg pageviews/hour (until certain date)
- number of days/week with pageviews
- number of hours per day with pageview
- (hour of the day, day of the week.. could require timezone info of the student...)
Need to extract relevant date for each course (mid-way point for example, due date of assignment 4.1 

In [37]:
pd.DataFrame.from_dict(r['participations']).head()

,created_at,url
0,2020-08-05T14:32:10Z,https://student.emeritus.org/courses/1574/quiz...
1,2020-08-05T14:32:34Z,https://student.emeritus.org/courses/1574/quiz...
2,2020-08-05T14:49:26Z,https://student.emeritus.org/api/v1/courses/15...
3,2020-08-10T18:23:03Z,https://student.emeritus.org/api/v1/courses/15...
4,2020-08-13T12:46:09Z,https://student.emeritus.org/api/v1/courses/15...


From the partecipations we MIGHT be able to create these features:
- time between publish date and participation
- time before or after deadline
- partecipations /day
- number of days with parteciaptions
- number of partecipations/hour
- number of hours with partecipations
- (hour of the day, day of the week.. could require timezone info of the student...)

### Get student outcomes

In [38]:
#get sections
course_id = 1574
                                      
r = my_request('/api/v1/courses/{}/sections?per_page=200'.format(course_id))

In [39]:
pd.DataFrame.from_dict(r)[['id','name']]

,id,name
0,4341,Berkeley Haas Artificial Intelligence
1,4606,Deferment
2,4607,Observers
3,4608,Section A
4,4609,Section B
5,4610,Section C
6,4604,Support
7,4605,Withdrawal


## Extract Valid assignments

In [53]:
# get assignment id (contains all sections)
course_id = '1574'
                                      
r = my_request('/api/v1/courses/{}/assignments?per_page=200'.format(course_id))

In [99]:
all_assignments = pd.DataFrame(r)
all_assignments.columns

Index(['id', 'description', 'due_at', 'unlock_at', 'lock_at',
       'points_possible', 'grading_type', 'assignment_group_id',
       'grading_standard_id', 'created_at', 'updated_at', 'peer_reviews',
       'automatic_peer_reviews', 'position',
       'grade_group_students_individually', 'anonymous_peer_reviews',
       'group_category_id', 'post_to_sis', 'moderated_grading',
       'omit_from_final_grade', 'intra_group_peer_reviews',
       'anonymous_instructor_annotations', 'anonymous_grading',
       'graders_anonymous_to_graders', 'grader_count',
       'grader_comments_visible_to_graders', 'final_grader_id',
       'grader_names_visible_to_final_grader', 'allowed_attempts',
       'secure_params', 'course_id', 'name', 'submission_types',
       'has_submitted_submissions', 'due_date_required', 'max_name_length',
       'in_closed_grading_period', 'is_quiz_assignment', 'can_duplicate',
       'original_course_id', 'original_assignment_id',
       'original_assignment_name', 'orig

In [104]:
#discussion_topic_posted_at
all_assignments.discussion_topic[1]

{'id': 93597,
 'title': 'Assignment 3.1 - Neural Networks – Teachable Machine - Section A',
 'last_reply_at': '2020-09-07T14:04:14Z',
 'created_at': '2020-07-28T13:36:28Z',
 'delayed_post_at': None,
 'posted_at': '2020-08-21T11:32:54Z',
 'assignment_id': 63393,
 'root_topic_id': None,
 'position': None,
 'podcast_has_student_posts': False,
 'discussion_type': 'threaded',
 'lock_at': None,
 'allow_rating': True,
 'only_graders_can_rate': False,
 'sort_by_rating': False,
 'is_section_specific': False,
 'user_name': None,
 'discussion_subentry_count': 20,
 'permissions': {'attach': True,
  'update': True,
  'reply': True,
  'delete': True},
 'require_initial_post': None,
 'user_can_see_posts': True,
 'podcast_url': None,
 'read_state': 'read',
 'unread_count': 19,
 'subscribed': False,
 'attachments': [],
 'published': True,
 'can_unpublish': False,
 'locked': False,
 'can_lock': True,
 'comments_disabled': False,
 'author': {},
 'html_url': 'https://student.emeritus.org/courses/1574/disc

In [50]:
# required_assignments = all_assignments[all_assignments.name.str.contains('Assignment')
#                              ^all_assignments.name.str.contains('Capstone')]
# required_assignments[['id','name']].head()

In [51]:
# capstones = all_assignments[all_assignments.name.str.contains('Capstone')]
# capstones[['id','name']].head()

## Assignments per student

In [44]:
#get list of assignments per student
course_id = '1574'
student_id = '71164'

r = my_request('/api/v1/courses/{}/students/submissions?student_ids[]={}&per_page=200'.format(course_id,student_id))

In [47]:
#pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']]

In [46]:
student_submissions = pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']]
student_submissions['grade'].replace(to_replace='complete',value=1, inplace=True)
student_submissions['grade'].fillna(0,inplace=True)
student_submissions['grade'] = student_submissions['grade'].astype(int) #convert boolean to int
student_submissions['late'] = student_submissions['late'].astype(int)
student_submissions['missing'] = student_submissions['missing'].astype(int)
student_submissions

,grade,assignment_id,late,missing,seconds_late
0,1,63387,0,0,0
1,1,63388,0,0,0
2,1,63389,0,0,0
3,1,63390,0,0,0
4,1,63391,0,0,0
5,1,63392,0,0,0
6,1,63393,0,0,0
7,1,63396,1,0,1205
8,0,63676,0,1,10247718
9,0,63679,0,1,9038118


This student submission could be transposed to create one feature per assignment id (match assignment id with name to merge across many courses).

## Debugging only

In [ ]:
course_id = '1116'
r = my_request('/api/v1/courses/{}/assignments?per_page=200'.format(course_id))
all_assignments = pd.DataFrame(r)


In [ ]:
for n in all_assignments.name:
        print(n)

In [ ]:
required_assignments = all_assignments[all_assignments.name.str.contains('Assignment')]
required_assignments = required_assignments[~required_assignments.name.str.contains('Capstone')]
capstones = all_assignments[all_assignments.name.str.contains('Capstone')]

In [ ]:
required_assignments = required_assignments[['id','name']]
capstones = capstones[['id','name']][['id','name']]

In [ ]:
required_assignments

In [ ]:
capstones

In [ ]:
student_ids = get_list_of_students(course_id)

In [ ]:
s_id = student_ids[0]

In [ ]:
check_student_passing_criteria(s_id, course_id, required_assignments, capstones)

In [ ]:
student_id =  s_id
r = my_request('/api/v1/courses/{}/students/submissions?student_ids[]={}&per_page=200'.format(course_id,student_id))
student_submissions = pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']] #could reduce to 'assignment_id' and 'grade'
student_submissions['grade'].replace(to_replace='complete',value=1, inplace=True)
student_submissions['grade'].replace(to_replace='incomplete',value=0, inplace=True)
student_submissions['grade'].fillna(0,inplace=True)
student_submissions['grade'] = student_submissions['grade'].astype(int) #convert boolean to int
#student_submissions['late'] = student_submissions['late'].astype(int) # not needed as of right now. could check if too late...
#student_submissions['missing'] = student_submissions['missing'].astype(int)

capstone_check = pd.merge(student_submissions, capstones, left_on='assignment_id', right_on='id', how='inner')['grade'].sum()

In [ ]:
outcome = check_student_passing_criteria(s_id, course_id, required_assignments, capstones)

In [ ]:
outcomes.append({'student_id' : s_id, 'course_id':course_id, 'outcome':outcome},ignore_index=True)

In [ ]:
outcomes